In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Simulate Data


Given a field name, and first 2 dimensional numbes, simulate tensor.

In [2]:

from fieldnn.utils.simulate import get_next_info, get_simulated_tensor_from_fldname

# Holder

1. Current Layer:
```python
leng_mask = info_idx == 0 # or info_idx != 0
leng = leng_mask.sum(-1)
```

2. Transfer

```python
old_leng = leng
```


3. Next Layer
```python
leng_mask = old_leng != 0
leng = (leng_mask == 0).sum(-1)
```

In [3]:
# leng_mask = info_idx == 0

import torch

def get_Layer2Holder(fullname, holder, Ignore_PSN_Layers = ['B', 'P']):
    # holder = holder
    d = {}
    for layername in list(reversed(fullname.split('-'))):
        if layername in Ignore_PSN_Layers: continue
        leng_mask = holder == 0
        leng = (leng_mask == 0).sum(-1)
        psn_idx = (leng_mask == False).cumsum(-1).masked_fill(leng_mask, 0)
        d[layername] = {'holder': holder, 
                        'leng_mask': leng_mask, 
                        'leng': leng, 
                        'psn_idx': psn_idx}
        # d[layername] = holder, psn_idx
        holder = leng
    Layer2Hoder = d
    return Layer2Hoder

In [4]:

def align_psn_idx(source_layer, current_layer, Layer2Idx, Layer2Holder):
    if source_layer == current_layer:
        psn_idx = Layer2Holder[current_layer]['psn_idx']
        return psn_idx
    else:
        source_psn_idx = Layer2Holder[source_layer]['psn_idx']
        current_leng_mask = Layer2Holder[current_layer]['leng_mask']
        gaps = Layer2Idx[current_layer] - Layer2Idx[source_layer]
        # print(gaps)
        # print(layername)
        # print(prev_info.shape)
        # print(leng_mask.shape)
        # print(leng.shape)
        # print(psn_idx.shape)
        shape0 = list(source_psn_idx.shape) + [1] * gaps
        shape1 = current_leng_mask.shape
        psn_idx = source_psn_idx.view(*shape0).expand(shape1).masked_fill(current_leng_mask, 0)
        # print(cpsn_idx.shape)
        return psn_idx

# Simulate Tensor

In [5]:
# import torch
# # ======= within forward

# ###############
# B_lenP = 3
# B2P_lnEC = [6, 5, 2] # 
# prefix_layers_num = 2
# vocab_size = 5001
# Ignore_PSN_Layers = ['B', 'P']
# ###############


# full_recfldgrn = 'B-P-EC-Diag:Value-DiagVdftGrn' # full_recflgrn.

# info_idx = get_simulated_tensor_from_fldname(full_recfldgrn, 
#                                              B_lenP, B2P_lnEC, 
#                                              prefix_layers_num, vocab_size)
# print(info_idx.shape)
# holder = torch.LongTensor(info_idx)


# ############### gsn_embeddings
# Layer2Idx = {v:idx for idx, v in enumerate(full_recfldgrn.split('-'))}
# name = full_recfldgrn.split('-')[-1]
# Layer2Holder = get_Layer2Holder(full_recfldgrn, holder, Ignore_PSN_Layers)
# psn_layers = list(reversed([i for i in Layer2Idx if i not in Ignore_PSN_Layers]))


# print(Layer2Idx)
# print(name)
# print([i for i in Layer2Holder], '<---- Layer2Holder')
# print(psn_layers, '<---- psn_layers')

# for source_layer in psn_layers:
#     cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
#     print(source_layer, cpsn_idx.shape)

# Real Tensor

In [7]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy


###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

In [8]:
recfldgrn_list = [
                  'A1C@V-A1CNumeDftGrn',
                  'Diag@DT-DTDftGrn',
                  'Diag@Value-DiagDftGrn',
                  'PNSectSent@Sentence-Tk@TknzLLMGrn'
                ]

In [9]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


## Analyze One RecFldGrn

In [10]:
full_recfldgrn = 'B-P-EC-Diag@Value-DiagDftGrn_idx'
info_idx = batch_rfg[full_recfldgrn]
print(info_idx.shape)
holder = torch.LongTensor(info_idx)

(4, 25, 22, 3)


In [12]:
# full_recfldgrn

def parse_full_recfldgrn(full_recfldgrn):
    # suffix = full_recfldgrn.split('_')[-1]
    recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    rec, fld = recfld.split('@')
    grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    grn, suffix = grn_suffix.split('_')
    prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    return prefix_ids, rec, fld, grn, suffix


prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn
print(prefix_ids)
print(rec)
print(fld)
print(grn)
print(suffix)
print(recfldgrn)

['B', 'P', 'EC']
Diag
Value
DiagDftGrn
idx
Diag@Value-DiagDftGrn


# Expander Core

Expander try to convert GRN_idx (or GRN_wgt) to embedding vectors.

It is from idx to vector.

It is a type of order-increase. 


Here the core include: CateEmbedding, NumeEmbedding, and LLMEmbed. 



## CateEmbedding (Expander)

### Module

In [11]:
# fieldlm.nn.embedding
import os
import torch
import numpy as np

class CatEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 vocab_size, 
                 embedding_size, 
                 init = 'random', 
                 freeze = False):
        
        super(CatEmbeddingLayer, self).__init__()
        
        # create embedding
        if init == 'random':
            # c. initial from random initialization
            self.embedding = torch.nn.Embedding(vocab_size, embedding_size, padding_idx = 0)
        
        elif type(init) == np.ndarray:
            # a. load from pretrained array. Here init is an array.
            weight = torch.FloatTensor(init)
            assert weight.shape == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
            
        elif os.path.isfile(init):
            # b. load from the pretrained array file.
            weight = torch.FloatTensor(np.load(init))
            assert tuple(weight.shape) == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
        
        else:
            raise ValueError(f'In correct init method "{init}"')
        
    def forward(self, holder):
        # info is the grain
        info = self.embedding(holder)
        return info

### Config

In [12]:
### Main Config
### others will change with each recfldgrn

#######################
embed_size = 512
init = 'random'
#######################

In [13]:


def get_recfldgrn_embedding(full_recfldgrn, embed_size, vocab_size, init = 'random'):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'vocab_size': vocab_size}
    return embed_para


### Usage 

#### recfldgrn information

In [14]:
import pandas as pd

# (1) full recfldgrn information
full_recfldgrn = 'B-P-EC-Diag@Value-DiagDftGrn_idx'

prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn

In [15]:
# (2) get vocab information
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
# TODO: also adding padding to v2idx
vocab_size = len(v2idx)
print(vocab_size)

801


#### get configuration

In [16]:
embed_para = get_recfldgrn_embedding(recfldgrn, embed_size, vocab_size, init)
print(embed_para)

{'embedding_size': 512, 'init': 'random', 'vocab_size': 801}


#### init model

In [17]:
if suffix == 'idx':
    NN = CatEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

CatEmbeddingLayer(
  (embedding): Embedding(801, 512, padding_idx=0)
)


#### prepare input

In [18]:


# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape)

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')


(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information


#### I-NN-O

In [19]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info = info * holder_wgt.unsqueeze(-1)
    print(info.shape, '<----- updated by holder_wgt')
    
    

torch.Size([4, 25, 22, 3]) <----- holder shape
torch.Size([4, 25, 22, 3, 512]) <----- info shape


## NumeEmbedding (Expander)

### Module

In [20]:
import os
import torch
import numpy as np

# just the same as the CateEmbeddingLayer

class NumEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 vocab_size, 
                 embedding_size, 
                 init = 'random', 
                 freeze = False):
        
        super(NumEmbeddingLayer, self).__init__()
        
        # create embedding
        if init == 'random':
            # c. initial from random initialization
            self.embedding = torch.nn.Embedding(vocab_size, embedding_size, padding_idx = 0)
        
        elif type(init) == np.ndarray:
            # a. load from pretrained array. Here init is an array.
            weight = torch.FloatTensor(init)
            assert weight.shape == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
            
        elif os.path.isfile(init):
            # b. load from the pretrained array file.
            weight = torch.FloatTensor(np.load(init))
            assert tuple(weight.shape) == (vocab_size, embedding_size)
            self.embedding = torch.nn.Embedding.from_pretrained(weight, freeze = freeze)
        
        else:
            raise ValueError(f'In correct init method "{init}"')
        
    def forward(self, holder):
        # info is the grain
        info = self.embedding(holder)
        return info

### Config

In [21]:
# same as CateEmbedding

### Usage

#### recfldgrn information

In [22]:
# (1) full recfldgrn information
full_recfldgrn = 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt'
prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn


# (2) get vocab information
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
# TODO: also adding padding to v2idx
vocab_size = len(v2idx)
print(vocab_size)

37


#### get configuration

In [23]:
embed_para = get_recfldgrn_embedding(recfldgrn, embed_size, vocab_size, init)
print(embed_para)

{'embedding_size': 512, 'init': 'random', 'vocab_size': 37}


#### init model

In [24]:
if suffix == 'idx':
    NN = CatEmbeddingLayer(**embed_para)
    print(NN)
elif suffix == 'wgt':
    NN = NumEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

NumEmbeddingLayer(
  (embedding): Embedding(37, 512, padding_idx=0)
)


#### prepare input

In [25]:
# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape)

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')


(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


In [26]:
holder[0, 0, 0]

tensor([ 0,  0,  0,  0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         0])

In [27]:
 holder_wgt[0, 0, 0]

tensor([ 0.,  0.,  0.,  0., 11.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.])

#### I-NN-O

In [28]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info_wgt = info * holder_wgt.unsqueeze(-1)
    print(info_wgt.shape, '<----- updated by holder_wgt')

torch.Size([4, 25, 1, 37]) <----- holder shape
torch.Size([4, 25, 1, 37, 512]) <----- info shape
torch.Size([4, 25, 1, 37, 512]) <----- updated by holder_wgt


In [29]:
holder[0, 0, 0]

tensor([ 0,  0,  0,  0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         0])

In [30]:
holder_wgt[0, 0, 0]

tensor([ 0.,  0.,  0.,  0., 11.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.])

In [31]:
info[0, 0, 0][:, 0]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.8789,  0.2751,  0.1745, -1.1499,
         0.7458, -0.1231,  1.2537, -0.6791, -1.9465, -0.7298,  0.6623,  1.0437,
        -0.1583, -0.7005,  0.1771,  0.3816,  0.8535,  1.0558,  1.3356, -0.5503,
        -0.5504, -0.4645, -1.2046,  1.3446, -1.3729,  1.0688,  1.5735,  1.2675,
        -1.7918,  0.2977,  0.7187, -0.7210,  0.0000],
       grad_fn=<SelectBackward0>)

In [32]:
info_wgt[0, 0, 0][:, 0]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  9.6680,  0.2751,  0.1745, -1.1499,
         0.7458, -0.1231,  1.2537, -0.6791, -1.9465, -0.7298,  0.6623,  1.0437,
        -0.1583, -0.7005,  0.1771,  0.3816,  0.8535,  1.0558,  1.3356, -0.5503,
        -0.5504, -0.4645, -1.2046,  1.3446, -1.3729,  1.0688,  1.5735,  1.2675,
        -1.7918,  0.2977,  0.7187, -0.7210,  0.0000],
       grad_fn=<SelectBackward0>)

## LLMEmbedding (Expander)

### Module

In [33]:
import os
import torch
import numpy as np
# from ...utils.layerfn import orderSeq, restoreSeq
from fieldnn.utils.layerfn import orderSeq, restoreSeq
from transformers import AutoModel


class LLMEmbeddingLayer(torch.nn.Module):

    def __init__(self, 
                 tokenizer, 
                 embedding_size, 
                 init, 
                 freeze = False):
        
        super(LLMEmbeddingLayer, self).__init__()
        
        self.tokenizer = tokenizer
        assert init == tokenizer.name_or_path 
        
        self.LLM = AutoModel.from_pretrained(init)
        self.hidden_size = self.LLM.config.hidden_size
        self.embedding_size = embedding_size
        
        self.linear  = torch.nn.Linear(self.hidden_size, self.embedding_size)
        self.init_linear_weights()
            
    def init_linear_weights(self):
        initrange = 0.1
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)
        
        
    def forward(self, holder):
        # 1. get leng_mask
        leng_mask = holder == 0
        
        # 2. get ordered holder
        ord_holder, ord_leng_mask, r_idx = self.reshape(holder, leng_mask)
        
        # 3. embedding ordered holder by LLM
        # expanding by the HuggingFace Language Model
        
        # 3.1 we might want to freeze LLM here
        output = self.LLM(ord_holder)
        # 3.2 adjust the hidden dimension
        ord_info_output = output['last_hidden_state']
        ord_info_output = self.linear(ord_info_output) # bias might not be zeros.
        ord_info_output = ord_info_output.masked_fill(ord_leng_mask.unsqueeze(-1), 0)
    
        # 4. restore orderded output to original shape
        info = self.restore(ord_info_output, leng_mask, r_idx)
        
        return info
    
    
    def reshape(self, holder, leng_mask):
        nbs = np.array(holder.shape[:-1]).prod()
        ngrn = holder.shape[-1]
        # print(nbs, ngrn, dim)
        
        tmp_holder = holder.contiguous().view(nbs, ngrn)
        # print(tmp_info.shape)

        tmp_leng_mask = leng_mask.contiguous().view(nbs, ngrn)
        # print(tmp_leng_mask.shape)

        tmp_leng = (tmp_leng_mask == 0).sum(-1)
        # print(tmp_leng.shape)
        
        ord_holder,    ord_leng, r_idx = orderSeq(tmp_holder, tmp_leng)
        ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
        return ord_holder, ord_leng_mask, r_idx
    
    
    
    def restore(self, ord_info_output, leng_mask, r_idx):
        info_new = restoreSeq(ord_info_output, r_idx)
        output_size = info_new.shape[-1]
        info_output = info_new.view(*list(leng_mask.shape) + [output_size])
        return info_output
        

### Config

In [34]:
def get_recfldgrn_llmembedding(full_recfldgrn, embed_size, tokenizer, init):
    embed_para =  {'embedding_size': embed_size,
                   'init': init, 
                   'tokenizer': tokenizer}
    return embed_para


### Usage

#### recfldgrn information

In [35]:
[i for i in batch_rfg]

['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [36]:
# (1) full recfldgrn information
full_recfldgrn = 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx'
prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
recfldgrn = rec + '@' + fld + '-' + grn


# (2) get vocab information
fldgrn_folder = 'data/ProcData/FldGrnInfo'
fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

if 'LLM' in full_recfldgrn:
    tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
    # TODO: also adding padding to v2idx
    vocab_size = len(tokenizer)
    print(vocab_size)
else:
    v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
    # TODO: also adding padding to v2idx
    vocab_size = len(v2idx)
    print(vocab_size)

30522


#### get configuration

In [37]:
init = tokenizer.name_or_path
embed_para = get_recfldgrn_llmembedding(recfldgrn, embed_size, tokenizer, init)
embed_para

{'embedding_size': 512,
 'init': 'bert-base-uncased',
 'tokenizer': BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})}

#### init model

In [38]:
if suffix == 'idx' and 'LLM' not in recfldgrn:
    NN = CatEmbeddingLayer(**embed_para)
    print(NN)
    
elif suffix == 'idx' and 'LLM' in recfldgrn:
    NN = LLMEmbeddingLayer(**embed_para)
    
elif suffix == 'wgt':
    NN = NumEmbeddingLayer(**embed_para)
    print(NN)
else:
    raise ValueError(f'suffix is not correct "{suffix}"')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### prepare input

In [39]:
# (1) get the info_raw from batch_rfg
info_raw = batch_rfg[full_recfldgrn]
print(info_raw.shape)

# (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
if suffix == 'idx':
    holder_wgt = 'Empty'
    holder = torch.LongTensor(info_raw)
elif suffix == 'wgt':
    holder_wgt = torch.FloatTensor(info_raw)
    # ATTENTION: here holder_wgt could contain zeros in some valid positions.
    holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
else:
    raise ValueError(f'Invalid suffix "{suffix}"')
    
print(holder.shape, '<------ holder shape')
if type(holder_wgt) == str: 
    print(holder_wgt, '<------ holder wgt information')
else:
    print(holder_wgt.shape, '<------ holder wgt information')


(4, 25, 1, 14, 121, 11)
torch.Size([4, 25, 1, 14, 121, 11]) <------ holder shape
Empty <------ holder wgt information


#### I-NN-O

In [40]:
print(holder.shape, '<----- holder shape')
info = NN(holder)
print(info.shape, '<----- info shape')

if type(holder_wgt) != str:
    info_wgt = info * holder_wgt.unsqueeze(-1)
    print(info_wgt.shape, '<----- updated by holder_wgt')

torch.Size([4, 25, 1, 14, 121, 11]) <----- holder shape
torch.Size([4, 25, 1, 14, 121, 11, 512]) <----- info shape


In [41]:
leng_mask = holder == 0
leng_mask.shape

torch.Size([4, 25, 1, 14, 121, 11])

In [42]:
info.shape

torch.Size([4, 25, 1, 14, 121, 11, 512])

In [43]:
# TODO: do the tensor validation

# Expander

## Module

In [45]:
import torch
import torch.nn.functional as F

from fieldnn.nn.embedding.catembedding import CatEmbeddingLayer
from fieldnn.nn.embedding.numembedding import NumEmbeddingLayer 
from fieldnn.nn.embedding.llmembedding import LLMEmbeddingLayer

# from ..nn.embedding.catembedding import CatEmbeddingLayer
# from ..nn.embedding.numembedding import NumEmbeddingLayer 
# from ..nn.embedding.llmembedding import LLMEmbeddingLayer


class Expander_Layer(torch.nn.Module):
    
    '''Only for Increasing Input_idx's Order'''
    def __init__(self, full_recfldgrn, output_recfld, expander_para):
        super(Expander_Layer, self).__init__()
        
        self.input_size = expander_para['input_size']
        self.output_size = expander_para['output_size']
        
        # Part 1: embedding
        self.input_fullname = full_recfldgrn
        self.output_fullname = output_recfld
        
        assert 'Grn' in full_recfldgrn
        assert full_recfldgrn.split('Grn')[0] == output_recfld
        
        nn_name, embed_para = expander_para[self.input_fullname]
        
        if nn_name.lower() == 'catembeddinglayer':
            assert 'idx' in full_recfldgrn and 'LLM' not in full_recfldgrn
            self.Embed = CatEmbeddingLayer(**embed_para)
        elif nn_name.lower() == 'llmembeddinglayer':
            assert 'idx' in full_recfldgrn and 'LLM' in full_recfldgrn
            self.Embed = LLMEmbeddingLayer(**embed_para)
        elif nn_name.lower() == 'numembeddinglayer':
            assert 'wgt' in full_recfldgrn
            self.Embed = NumEmbeddingLayer(**embed_para)
        else:
            raise ValueError(f'suffix is not correct "{full_recfldgrn}"')

        self.embed_size = self.output_size
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in expander_para['postprocess'].items():
            if method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.embed_size, **config)

    def forward(self, fullname, holder, holder_wgt = 'Empty'):
        '''
            fullname: full name of field, GRN is here
            holder: info_idx
            info: info_wgt
        '''
        assert self.input_fullname == fullname
        leng_mask = holder == 0
        embed = self.Embed(holder)
        
        if type(holder_wgt) != str:
            embed = embed * holder_wgt.unsqueeze(-1)
        
        for nn, layer in self.postprocess.items():
            embed = layer(embed)
        
        return self.output_fullname, holder, embed

## Config

In [46]:
# def get_recfldgrn_embedding(full_recfldgrn, embed_size, vocab_size, init = 'random'):
#     embed_para =  {'embedding_size': embed_size,
#                    'init': init, 
#                    'vocab_size': vocab_size}
#     return embed_para

# def get_recfldgrn_llmembedding(full_recfldgrn, embed_size, tokenizer, init):
#     embed_para =  {'embedding_size': embed_size,
#                    'init': init, 
#                    'tokenizer': tokenizer}
#     return embed_para

In [47]:
def get_expander_para(full_recfldgrn, 
                      nn_name,
                      embed_size, vocabsize_tokenizer, init, 
                      postprocess):
    
    expander_para = {}
    #(0) Input size, output size
    expander_para['input_size'] = None
    expander_para['output_size'] = embed_size
    expander_para['nn_name'] = nn_name

    #(1) Main NN
    # print(nn_name)
    if nn_name.lower() == 'catembeddinglayer':
        para = get_recfldgrn_embedding(full_recfldgrn, embed_size, vocabsize_tokenizer, init)
    elif nn_name.lower() == 'numembeddinglayer':
        para = get_recfldgrn_embedding(full_recfldgrn, embed_size, vocabsize_tokenizer, init)
    elif nn_name.lower() == 'llmembeddinglayer':
        para = get_recfldgrn_llmembedding(full_recfldgrn, embed_size,  vocabsize_tokenizer, init)
    else:
        raise ValueError(f'The NN "{nn_name}" is not available yet')
    
    # print(para, '<--------- inside get_expander_para')
    expander_para[full_recfldgrn] = nn_name, para

    # (2) Post Process
    expander_para['postprocess'] = postprocess
    
    return expander_para

## Usage

### recfldgrn information

In [48]:
full_recfldgrn_list = ['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
                       'B-P-EC-Diag@Value-DiagDftGrn_idx',
                       'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

full_recfldgrn_list

['B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [49]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [50]:
full_rfg_to_basicInfo = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) full recfldgrn information
    prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
    recfldgrn = rec + '@' + fld + '-' + grn


    # (2) get vocab information
    fldgrn_folder = 'data/ProcData/FldGrnInfo'
    fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
    df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

    if 'LLM' in full_recfldgrn:
        tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # TODO: also adding padding to v2idx
        # vocab_size = len(tokenizer)
        # print(vocab_size)
        vocab_tokenizer = tokenizer
        init = tokenizer.name_or_path
    else:
        v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # TODO: also adding padding to v2idx
        vocab_size = len(v2idx)
        # print(vocab_size)
        vocab_tokenizer = vocab_size
        init = 'random'
    
    d = {'vocabsize_tokenizer': vocab_tokenizer, 'init': init}
    full_rfg_to_basicInfo[full_recfldgrn] = d
    
    
full_rfg_to_basicInfo

{'B-P-EC-A1C@V-A1CNumeDftGrn_wgt': {'vocabsize_tokenizer': 37,
  'init': 'random'},
 'B-P-EC-Diag@Value-DiagDftGrn_idx': {'vocabsize_tokenizer': 801,
  'init': 'random'},
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx': {'vocabsize_tokenizer': BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
  'init': 'bert-base-uncased'}}

### get configuration

In [51]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


full_rfg_to_ExpanderConfig = {}

for full_recfldgrn in full_recfldgrn_list:
    
    # determine the nn_name
    if '_idx' in full_recfldgrn and 'LLM' not in full_recfldgrn:
        nn_name = 'CatEmbeddingLayer'

    elif '_idx' in full_recfldgrn and 'LLM' in full_recfldgrn:
        nn_name = 'LLMEmbeddingLayer'

    elif '_wgt' in full_recfldgrn:
        nn_name = 'NumEmbeddingLayer'
    else:
        raise ValueError(f'full_recfldgrn is not correct "{full_recfldgrn}"')
    
    d = full_rfg_to_basicInfo[full_recfldgrn]
    
    # print('\n----')
    # print(full_recfldgrn)
    # print(d)
    # print(nn_name)
    expander_para = get_expander_para(full_recfldgrn, 
                                       nn_name,
                                       embed_size, d['vocabsize_tokenizer'], d['init'], 
                                       process)
    # print(expander_para)
    output_recfld = full_recfldgrn.split('Grn')[0]
    
    full_rfg_to_ExpanderConfig[full_recfldgrn] = {
        'full_recfldgrn': full_recfldgrn,
        'output_recfld': output_recfld,
        'expander_para': expander_para,
    }
    
    
# full_rfg_to_ExpanderConfig

In [52]:
# full_rfg_to_ExpanderConfig

### init model

In [53]:
FullRFG_2_Embed = {}

for full_recfldgrn, ExpanderConfig in full_rfg_to_ExpanderConfig.items():
    
    FullRFG_2_Embed[full_recfldgrn] = Expander_Layer(**ExpanderConfig)
    print(full_recfldgrn)
    print(FullRFG_2_Embed[full_recfldgrn])
    

B-P-EC-A1C@V-A1CNumeDftGrn_wgt
Expander_Layer(
  (Embed): NumEmbeddingLayer(
    (embedding): Embedding(37, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC-Diag@Value-DiagDftGrn_idx
Expander_Layer(
  (Embed): CatEmbeddingLayer(
    (embedding): Embedding(801, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
Expander_Layer(
  (Embed): LLMEmbeddingLayer(
    (LLM): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense):

### prepare input

In [54]:
for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print(full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str: 
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')

    print('\n')

B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
(4, 25, 1, 14, 121, 11)
torch.Size([4, 25, 1, 14, 121, 11]) <------ holder shape
Empty <------ holder wgt information




### I-NN-O

In [55]:
for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print('Input:',full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str:  
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')
        
        
    NN = FullRFG_2_Embed[full_recfldgrn] 
    print(holder.shape, '<----- holder shape')
    output_recfld, holder, info = NN(full_recfldgrn, holder, holder_wgt)
    print('Output:', output_recfld)
    print(info.shape, '<----- info shape')

    print('\n')

Input: B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information
torch.Size([4, 25, 1, 37]) <----- holder shape
Output: B-P-EC-A1C@V-A1CNumeDft
torch.Size([4, 25, 1, 37, 128]) <----- info shape


Input: B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 22, 3]) <----- holder shape
Output: B-P-EC-Diag@Value-DiagDft
torch.Size([4, 25, 22, 3, 128]) <----- info shape


Input: B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
(4, 25, 1, 14, 121, 11)
torch.Size([4, 25, 1, 14, 121, 11]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 1, 14, 121, 11]) <----- holder shape
Output: B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM
torch.Size([4, 25, 1, 14, 121, 11, 128]) <----- info shape


